# AI Travel Agent Setup

This notebook sets up and runs the AI Travel Agent application in Google Colab. Follow the steps below:

1. Install dependencies.
2. Set environment variables for SerpAPI and SendGrid.
3. Write the `travel_app.py` script to disk.
4. Launch the Streamlit app and open it via a public URL.


In [ ]:
!pip install python-dotenv langchain-core langchain-openai langgraph sendgrid serpapi streamlit nbformat


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.1/102.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216

## 2. Set Environment Variables

Replace the placeholder strings with your actual API keys and verified email addresses. Run the following cell to set the environment variables.


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""


# Replace the following placeholder values with your own credentials
os.environ["SERPAPI_API_KEY"] = ""
os.environ["SENDGRID_API_KEY"] = ""
os.environ["FROM_EMAIL"] = "raj.dandekar8@gmail.com"
os.environ["TO_EMAIL"] = "hello@vizuara.com"
os.environ["EMAIL_SUBJECT"] = "Travel Information"


## 3. Write the `travel_app.py` Script

The following cell writes the complete `travel_app.py` file to `/mnt/data/travel_app.py`. This script contains the Streamlit application, the agent logic, and the tool definitions.


In [ ]:

# pylint: disable = http-used,print-used,no-self-use,invalid-name
import datetime
import operator
import os
import uuid

from typing import Annotated, TypedDict, Optional
from dotenv import load_dotenv
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail
import serpapi

# Tool definitions
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool

# Load environment variables from a .env file
_ = load_dotenv()

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
CURRENT_YEAR = datetime.datetime.now().year


In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]


TOOLS_SYSTEM_PROMPT = f"""You are a smart travel agency. Use the tools to look up information.
You are allowed to make multiple calls (either together or in sequence).
Only look up information when you are sure of what you want.
The current year is {CURRENT_YEAR}. If you don't get flights, try searching till you find them!
If you need to look up some information before asking a follow up question, you are allowed to do that!
In your output, include links to hotel websites and flight websites (if possible),
the logo of the hotel and the logo of the airline company (if possible),
and always include both the price of the flight and the price of the hotel (with currency).
For example, for hotels:
    Rate: $581 per night
    Total: $3,488
"""

# ----- Flights tool -----
class FlightsInput(BaseModel):
    departure_airport: Optional[str] = Field(description='Departure airport code (IATA)')
    arrival_airport: Optional[str]   = Field(description='Arrival airport code (IATA)')
    outbound_date: Optional[str]      = Field(description='Outbound date (YYYY-MM-DD)')
    return_date: Optional[str]        = Field(description='Return date (YYYY-MM-DD)')
    adults: Optional[int] = Field(1, description='Number of adults (default 1)')
    children: Optional[int] = Field(0, description='Number of children (default 0)')
    infants_in_seat: Optional[int] = Field(0, description='Number of infants in seat (default 0)')
    infants_on_lap: Optional[int]   = Field(0, description='Number of infants on lap (default 0)')

class FlightsInputSchema(BaseModel):
    params: FlightsInput

@tool(args_schema=FlightsInputSchema)
def flights_finder(params: FlightsInput):
    """
    Find flights using the Google Flights engine (via SerpAPI).
    Returns:
        dict or str: Flight search results or error message.
    """
    query = {
        'api_key': os.environ.get('SERPAPI_API_KEY'),
        'engine': 'google_flights',
        'hl': 'en',
        'gl': 'us',
        'departure_id': params.departure_airport,
        'arrival_id': params.arrival_airport,
        'outbound_date': params.outbound_date,
        'return_date': params.return_date,
        'currency': 'USD',
        'adults': params.adults,
        'infants_in_seat': params.infants_in_seat,
        'stops': '1',
        'infants_on_lap': params.infants_on_lap,
        'children': params.children
    }
    try:
        search = serpapi.search(query)
        return search.data['best_flights']
    except Exception as e:
        return str(e)

In [ ]:
# ----- Hotels tool -----
class HotelsInput(BaseModel):
    q: str = Field(description='Location for hotels (e.g., "New York")')
    check_in_date: str  = Field(description='Check-in date (YYYY-MM-DD)')
    check_out_date: str = Field(description='Check-out date (YYYY-MM-DD)')
    sort_by: Optional[str] = Field(8, description='Sorting parameter (default=8 for rating)')
    adults: Optional[int]   = Field(1, description='Number of adults (default 1)')
    children: Optional[int] = Field(0, description='Number of children (default 0)')
    rooms: Optional[int]    = Field(1, description='Number of rooms (default 1)')
    hotel_class: Optional[str] = Field(None, description='Filter by hotel class (e.g., "3" or "4")')

class HotelsInputSchema(BaseModel):
    params: HotelsInput

@tool(args_schema=HotelsInputSchema)
def hotels_finder(params: HotelsInput):
    """
    Find hotels using the Google Hotels engine (via SerpAPI).
    Returns:
        list or str: Up to 5 hotel property dicts or error message.
    """
    query = {
        'api_key': os.environ.get('SERPAPI_API_KEY'),
        'engine': 'google_hotels',
        'hl': 'en',
        'gl': 'us',
        'q': params.q,
        'check_in_date': params.check_in_date,
        'check_out_date': params.check_out_date,
        'currency': 'USD',
        'adults': params.adults,
        'children': params.children,
        'rooms': params.rooms,
        'sort_by': params.sort_by,
        'hotel_class': params.hotel_class
    }
    try:
        search = serpapi.search(query)
        data = search.data
        return data['properties'][:5]
    except Exception as e:
        return str(e)


In [ ]:
TOOLS = [flights_finder, hotels_finder]

In [ ]:
EMAILS_SYSTEM_PROMPT = """Your task is to convert structured markdown-like text into a valid HTML email body.
- Do not include a ```html preamble in your response.
- The output should be proper HTML, ready to be used as an email body.
"""

In [ ]:

class Agent:
    def __init__(self):
        # Map name → tool function
        self._tools = {t.name: t for t in TOOLS}
        # A ChatOpenAI instance bound to both tools
        self._tools_llm = ChatOpenAI(model='gpt-4o').bind_tools(TOOLS)

        # Build the LangGraph state machine
        builder = StateGraph(AgentState)
        builder.add_node('call_tools_llm', self.call_tools_llm)
        builder.add_node('invoke_tools', self.invoke_tools)
        builder.add_node('email_sender', self.email_sender)
        builder.set_entry_point('call_tools_llm')

        # If the LLM's last message has no tool calls → go to email_sender
        builder.add_conditional_edges(
            'call_tools_llm',
            Agent.exists_action,
            {'more_tools': 'invoke_tools', 'email_sender': 'email_sender'}
        )
        builder.add_edge('invoke_tools', 'call_tools_llm')
        builder.add_edge('email_sender', END)

        memory = MemorySaver()
        self.graph = builder.compile(checkpointer=memory, interrupt_before=['email_sender'])

    @staticmethod
    def exists_action(state: AgentState):
        # If the last LLM response contains no tool_calls, move to email_sender
        result = state['messages'][-1]
        if len(result.tool_calls) == 0:
            return 'email_sender'
        return 'more_tools'

    def call_tools_llm(self, state: AgentState):
        # Prepend our system prompt, then ask the LLM to pick which tools to call
        messages = state['messages']
        messages = [SystemMessage(content=TOOLS_SYSTEM_PROMPT)] + messages
        message = self._tools_llm.invoke(messages)
        return {'messages': [message]}

    def invoke_tools(self, state: AgentState):
        # Execute each requested tool and wrap results in ToolMessages
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            if t['name'] not in self._tools:
                results.append(
                    ToolMessage(tool_call_id=t['id'], name=t['name'], content="Bad tool name")
                )
            else:
                result = self._tools[t['name']].invoke(t['args'])
                results.append(
                    ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result))
                )
        return {'messages': results}

    def email_sender(self, state: AgentState):
        # (We won't actually use this node for Gradio; see below.)
        return {'messages': []}


# ===== 3) Helper function to send email via SendGrid =====

def send_html_email(travel_html: str, sender: str, receiver: str, subject: str) -> str:
    """
    Uses SendGrid to send travel_html as the email body.
    Returns a status string.
    """
    message = Mail(
        from_email=sender,
        to_emails=receiver,
        subject=subject,
        html_content=travel_html
    )
    try:
        sg = SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
        resp = sg.send(message)
        return f"Email sent (status {resp.status_code})"
    except Exception as e:
        return f"Error sending email: {e}"


In [ ]:
# ===== 4) Instantiate a single global Agent =====

agent = Agent()

In [ ]:
# ===== 5) Gradio callables =====

def process_query_gradio(user_query: str) -> str:
    """
    Run the agent on the given travel query string, return the final text output.
    """
    thread_id = str(uuid.uuid4())
    # Create a single HumanMessage containing the entire query
    messages = [HumanMessage(content=user_query)]
    config = {'configurable': {'thread_id': thread_id}}

    result = agent.graph.invoke({'messages': messages}, config=config)
    # The agent’s final response (no further tool calls) is the last message’s .content
    return result['messages'][-1].content


In [ ]:
def process_email_gradio(travel_info: str, sender: str, receiver: str, subject: str) -> str:
    """
    Take the travel_info (HTML or plain text from the agent),
    plus sender/receiver/subject, and send via SendGrid.
    """
    if not sender or not receiver or not subject or not travel_info:
        return "Error: All fields are required."
    return send_html_email(travel_info, sender, receiver, subject)

In [ ]:
# ===== 6) Build the Gradio Interface =====
!pip install gradio
import gradio as gr
with gr.Blocks() as demo:
    gr.Markdown("# ✈️🌍 AI Travel Agent (Gradio Edition)")
    gr.Markdown("Enter a travel query below (e.g. “Flights from New York to London June 10–15, and 4-star hotels”).")

    # Textbox to accept the user’s travel query
    query_input = gr.Textbox(lines=3, placeholder="Type your travel query here…", label="Travel Query")
    query_button = gr.Button("Get Travel Information")
    travel_output = gr.Markdown("", label="Travel Info (Agent’s Response)")

    # When clicked, run process_query_gradio and show result in travel_output
    query_button.click(fn=process_query_gradio, inputs=query_input, outputs=travel_output)

    gr.Markdown("---\n## Send the Above Info via Email")
    gr.Markdown("Enter sender, receiver, and subject. The email body will be exactly what the agent printed above.")
    sender_input   = gr.Textbox(label="Sender Email")
    receiver_input = gr.Textbox(label="Receiver Email")
    subject_input  = gr.Textbox(label="Subject", value="Travel Information")
    email_button   = gr.Button("Send Email")
    email_status   = gr.Textbox(label="Email Status / Error")

    # When clicked, run process_email_gradio using travel_output plus the three email fields
    email_button.click(
        fn=process_email_gradio,
        inputs=[travel_output, sender_input, receiver_input, subject_input],
        outputs=email_status
    )

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 134.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.2 MB/s eta 0:00:00


In [ ]:
# Launch Gradio app
if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ec613affd346f76fcb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail

message = Mail(
    from_email="rajatdandekar@vizuara.com",
    to_emails="recipient@example.com",
    subject="SendGrid Permission Test",
    html_content="<strong>If you see this, permission is fixed!</strong>"
)

try:
    sg = SendGridAPIClient(os.environ["SENDGRID_API_KEY"])
    resp = sg.send(message)
    print("Response status:", resp.status_code)  # Expect 202, not 403
except Exception as e:
    print("SendGrid error:", e)


SendGrid error: HTTP Error 403: Forbidden
